# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [1]:
# Imports. Feel free to add or remove as necessary 

!pip install keras==2.9.0rc1
!pip install -q tensorflow_addons
# !pip install keras-cv-attention-models -q 

import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.9.0rc1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:00


In [2]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

Mounted at /content/drive


In [4]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git

Cloning into 'finnegan-paleo-ml'...
remote: Enumerating objects: 371, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 371 (delta 26), reused 28 (delta 16), pack-reused 326
Receiving objects: 100% (371/371), 188.18 KiB | 5.70 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [5]:
# @title Data stored in drive
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
# dir_code = '/content/finnegan-paleo-ml/Code'
dir_code = '/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code' #Have to override here because data can't be version controlled and is in drive
# dir_git_home = '/content/finnegan-paleo-ml'
dir_git_home = '/content/drive/MyDrive/MV1012_SBB_images/ML_projects/'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = dir_dataset_specs = f'{dir_ml_home}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [6]:
import pandas as pd
pd.read_csv(dir_labels_csv)

,sample_name,Broken,species,object_num,age
0,MV1012-BC-12,unbroken,NOT FORAM,1.0,1994.0
1,MV1012-BC-12,unbroken,suggrunda eckisi,2.0,1994.0
2,MV1012-BC-12,broken,bulimina exilis,3.0,1994.0
3,MV1012-BC-12,unbroken,nonionella stella,4.0,1994.0
4,MV1012-BC-12,unbroken,NOT FORAM,5.0,1994.0
...,...,...,...,...,...
12631,MV1012-BC-9-10,unbroken,nonionella stella,165.0,1998.4
12632,MV1012-BC-9-10,unbroken,nonionella stella,166.0,1998.4
12633,MV1012-BC-9-10,broken,NOT FORAM,167.0,1998.4
12634,MV1012-BC-9-10,unbroken,NOT FORAM,168.0,1998.4


In [7]:
os.chdir(dir_code)
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor
os.chdir(dir_git_home)

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [8]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [9]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df = labels_df[labels_df['Broken'].isin(['unbroken', 'broken'])]

labels_df.head()


Broken            species     age   test    val
sample_name  object_num                                                   
MV1012-BC-12 1.0         unbroken          NOT FORAM  1994.0  False  False
             2.0         unbroken   suggrunda eckisi  1994.0  False  False
             3.0           broken    bulimina exilis  1994.0  False  False
             4.0         unbroken  nonionella stella  1994.0  False  False
             5.0         unbroken          NOT FORAM  1994.0  False  False

In [10]:
labels_df[(labels_df['Broken'] == 'micro')]

,,Broken,species,age,test,val
sample_name,object_num,,,,,


In [11]:
%cd /content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code

/content/drive/.shortcut-targets-by-id/1pfS62oxpwDOc9RME825AUzAZ_qVVMlxG/MV1012_SBB_images/ML_projects/Code


In [12]:
%cd /content/drive/MyDrive/MV1012_SBB_images/ML_projects/Processed_data/

/content/drive/.shortcut-targets-by-id/1pfS62oxpwDOc9RME825AUzAZ_qVVMlxG/MV1012_SBB_images/ML_projects/Processed_data


In [13]:
dir_ml_home

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects'

In [ ]:
import shutil
for split in ['train', 'test', 'val']:
    shutil.rmtree(f'{dir_ml_home}/Processed_data/forams' + f'/{split}')
    os.mkdir(f'{dir_ml_home}/Processed_data/forams' + f'/{split}')
    


In [ ]:
git()

fatal: invalid reference: origin


In [14]:
#Run data processing
data_processor.run_processing(dir_processed_data, dir_raw_training_images, labels_df, verbose=True, override=False)

Streaming output truncated to the last 5000 lines.
skipped ('MV1012-BC-24', 466) becuase it is not in labels_df
skipped ('MV1012-BC-24', 719) becuase it is not in labels_df
skipped ('MV1012-BC-24', 749) becuase it is not in labels_df
skipped ('MV1012-BC-24', 407) becuase it is not in labels_df
skipped ('MV1012-BC-24', 261) becuase it is not in labels_df
skipped ('MV1012-BC-24', 680) becuase it is not in labels_df
skipped ('MV1012-BC-24', 622) becuase it is not in labels_df
skipped ('MV1012-BC-24', 262) becuase it is not in labels_df
skipped ('MV1012-BC-24', 240) becuase it is not in labels_df
skipped ('MV1012-BC-24', 350) becuase it is not in labels_df
skipped ('MV1012-BC-24', 755) becuase it is not in labels_df
skipped ('MV1012-BC-24', 767) becuase it is not in labels_df
skipped ('MV1012-BC-24', 312) becuase it is not in labels_df
skipped ('MV1012-BC-24', 327) becuase it is not in labels_df
skipped ('MV1012-BC-24', 307) becuase it is not in labels_df
skipped ('MV1012-BC-24', 282) becu

## Create tensorflow dataset and push to google cloud

In [ ]:
data_processor

<module 'data_processing.data_processor' from '/content/finnegan-paleo-ml/Code/data_processing/data_processor.py'>

In [11]:
import inspect
print(inspect.getsource(data_processor.create_cloud_dataset))

def create_cloud_dataset(dir_local_processed_data, dir_cloud_data, dir_dataset_specs, batch_size):
    # TODO: support for dataset data that's not in the same dir as specs
    cur_cwd = os.getcwd()
    os.chdir(dir_dataset_specs)
    import forams
    os.chdir(cur_cwd)
    builder_kwargs = {
        'image_label_dir': dir_local_processed_data,
    }
    training_set = tfds.load('forams', split='train', builder_kwargs=builder_kwargs)
    validation_set = tfds.load('forams', split='val', builder_kwargs=builder_kwargs)
    testing_set = tfds.load('forams', split='test', builder_kwargs=builder_kwargs)
    print(training_set)
    # os.chdir(cur_cwd)
    tf.data.experimental.save(training_set, f'{dir_cloud_data}/training{batch_size}')
    tf.data.experimental.save(validation_set, f'{dir_cloud_data}/validation{batch_size}')
    tf.data.experimental.save(testing_set, f'{dir_cloud_data}/testing{batch_size}')

    # Above code deprecated below not supported in colab default tf version
    # tf.d

In [12]:
dir_processed_data

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams'

In [20]:
!git branch

* main


In [19]:
git()

fatal: invalid reference: origin


In [16]:
data_processor.create_cloud_dataset(dir_processed_data, dir_google_cloud, dir_dataset_specs, 1234)

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/forams/1.0.0.incompleteHAJT7L/forams-train.tfrecord*...:   0%|          | …

Generating val examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/forams/1.0.0.incompleteHAJT7L/forams-val.tfrecord*...:   0%|          | 0/…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/forams/1.0.0.incompleteHAJT7L/forams-test.tfrecord*...:   0%|          | 0…

Dataset forams downloaded and prepared to /root/tensorflow_datasets/forams/1.0.0. Subsequent calls will reuse this data.


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


<_PrefetchDataset element_spec={'chamber_broken': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(224, 224, 3), dtype=tf.uint8, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [ ]:
training_set, validation_set, testing_set = data_processor.load_cloud_dataset(dir_google_cloud, 1024)

In [ ]:
for i in training_set:
    print(i)
    break

{'species': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'chamber_broken': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'image': <tf.Tensor: shape=(224, 224, 3), dtype=uint8, numpy=
array([[[34, 38, 37],
        [30, 34, 33],
        [21, 25, 24],
        ...,
        [44, 44, 44],
        [44, 44, 42],
        [50, 50, 48]],

       [[34, 38, 37],
        [30, 34, 33],
        [21, 25, 24],
        ...,
        [41, 41, 41],
        [40, 40, 38],
        [42, 42, 40]],

       [[32, 36, 35],
        [27, 31, 30],
        [20, 24, 23],
        ...,
        [37, 37, 37],
        [38, 38, 36],
        [35, 35, 33]],

       ...,

       [[27, 27, 27],
        [31, 31, 31],
        [34, 34, 34],
        ...,
        [39, 39, 39],
        [54, 54, 54],
        [65, 65, 65]],

       [[35, 35, 35],
        [39, 39, 39],
        [41, 41, 41],
        ...,
        [29, 29, 29],
        [49, 49, 49],
        [55, 55, 55]],

       [[34, 34, 34],
        [34, 34, 34],
        [35, 35, 35],
    